# GF-Math Tryouts


Addition and Subtraction in a Galois Field base 2, are implemented as modulo 2 additions/subtracions which means that we can implement this using "xor" for these operations. Additions and Subtractions in GF(2) are carryless operations.

In [ ]:
# x + y
def gf_add(x, y):
    return x ^ y

# x - y
def gf_sub(x, y):
    return x ^ y

Multiplication is based pon polynomial math

* $10001001 = (x^7 + x^3 + 1)$ = 128 + 9 = 137
* $00101010 = (x^5 + x^3 + x)$ = 32 + 8 + 2 = 42

Let's multiply this

* $(x^7 + x^3 + 1)(x^5 + x^3 + x) = x^7(x^5 + x^3 + x) + x^3(x^5 + x^3 + x) + 1(x^5 + x^3 + x)$
* $= (x^{12} + x^{10} + x^8) + (x^8 + x^6 + x^4) + (x^5 + x^3 + x)$
* $= x^{12} + x^{10} + 2x^8 + x^6 + x^5 + x^4 + x^3 + x$
* $= x^{12} + x^{10} + x^6 + x^5 + x^4 + x^3 + x$

Result in binary is

* $1010001111010$ = 4096 + 1024 + 64 + 32 + 16 + 8 + 2 = 5242


In [ ]:
# x * y
def gf_polymul(x,y):
    result = 0
    i=0
    while (y>>i) > 0:
        if y & (1<<i):
            result = gf_add(result,x<<i)
        i+=1
    return result

In [ ]:
gf_polymul(137,42)

The most significant bit of this polynomial is now at position 12 (because of teh term $x^{12}$), This exceedes the $GF(2^8)$, so we must somehow reduce this polyomial to the size of $GF(2^8)$

The result is reduced modulo $100011101$ (0x11d), which is a irreducible polynom. There exist more of these primitive irreducible polynomials for GF(2^8).

* $x^8 + x^4 + x^3 + x^2 + 1$
* $x^8 + x^5 + x^3 + x^1 + 1$
* $x^8 + x^6 + x^4 + x^3 + x^2 + x^1 + 1$
* $x^8 + x^6 + x^5 + x^1 + 1$
* $x^8 + x^6 + x^5 + x^2 + 1$
* $x^8 + x^6 + x^5 + x^3 + 1$
* $x^8 + x^7 + x^6 + x^1 + 1$
* $x^8 + x^7 + x^6 + x^5 + x^2 + x^1 + 1$

Next step is to divide the result of the multiplication using a irreducible polynomial.

<pre>
1010001111010 : 100011101 = 11000011
100011101
-------------
  10110101010
  100011101
  -----------
    111011110
    100011101
    ---------
     11000011
</pre

In [ ]:
def gf_msb(value):
    msb = 0
    while value >> msb:
        msb+=1
    return msb

def gf_reduce_div(dividend, divisor=None):
    dividend_msb = gf_msb(dividend)
    divisor_msb = gf_msb(divisor)
    
    if dividend_msb<divisor_msb :
        return dividend
    
    # for all bits above the divisor
    for i in range(dividend_msb-divisor_msb,-1,-1):
        # we check that particular bit whether it is set
        if dividend & (1<<(divisor_msb+i-1)):
            # if so we subtract the shifted divisor (via gf_add/gf_sub)
            # that will kill the upper most bit 
            dividend ^=divisor << i
    
    return dividend

# This is how we would do that in school, this is quite laborous
def gf_mul_reduce_div(x, y, d):
    result = gf_polymul(x,y)
    return gf_reduce_div(result,d)

gf_mul_reduce_div(137,42, 0x11d)

There are different methods of multiplying... $e^{log_e(x)} \cdot e^{log_e(y)}$ = $e^{log_e(x)+log_e(y)}$.

Using a proper "e", e.g. the generator number of the primitive generator polynomial, we can implement a multiplication using additions and look up tables for the log-value


If we work on this irreducible polynom 0x11d, we can calculate all exponentials and also all logarithms, within a certain range.

In [ ]:
gf_exp = [0] * 512 
gf_log = [0] * 256

def gf_init_exp_tables(prim_generator_polynomial=0x11d):
    global gf_exp, gf_log
    gf_exp = [0] * 512 
    gf_log = [0] * 256

    x=1
    for i in range(0,255):
        # compute "anti-log"
        gf_exp[i] = x
        # compute "log"
        gf_log[x] = i
        
        x = gf_mul_reduce_div(x,2,prim_generator_polynomial)

# x * y
def gf_mul_lut(x,y):
    if x==0 or y==0:
        return 0
    return gf_exp[(gf_log[x] + gf_log[y])%255]

gf_init_exp_tables(0x11d)
print(gf_mul_lut(137,42))

In [ ]:
# x / y
def gf_div_lut(x,y):
    if y==0:
        raise ZeroDivisionError()
    if x==0:
        return 0
    return gf_exp[ (255+gf_log[x] - gf_log[y])%255 ]

In [ ]:
# x ^ n  (n>0)
def gf_pow_lut(x, n):
    return gf_exp[(gf_log[x]*n)%255]

# 1 / x
def gf_inv_lut(x):
    return gf_exp[255-gf_log[x]]

In [ ]:
gf_pow_lut(2,8)